In [21]:
import arcpy, arcpy.mp
#from sys import argv
import time
import os
# import string, random

In [4]:
# Return list of workspaces in specified arcpy.env.workspace path
def listWorkSpaces(workPath):
    try:
        arcpy.env.workspace = workPath
        wsList = []
        ws = arcpy.ListWorkspaces("*", "FileGDB")
        for i,w in enumerate(ws):
            wsList.append(w)
            print(f'{i} - {w}')
        return wsList
    except:
        display(f'ERROR listing WorkSpaces in {workPath}')
        
# function to list all Feature Classes in specified FileGeoDatabase
def listItems(GDBpath):
    try:
        arcpy.env.workspace = GDBpath
        fcList = []
        fc = arcpy.ListFeatureClasses()
        print(f'Feature Classes/Shapefile in {GDBpath}: ')
        for i,f in enumerate(fc):
            fcList.append(f)
            print(f'{i} - {f}')
        return fcList
    except:
        display(f'ERROR listing Feature Classes in {GDBpath}')

# Delete feature class given path to GDB and fc name
def deleteFeatureClass(GDBpath,fcName):
    try:
        quest = (input(f'DELETE FEATURE CLASS - {GDBpath}\\{fcName} ??? - ') or 'NO').upper()
        if quest == 'YES':
            result = arcpy.Delete_management(f'{GDBpath}\\{fcName}')
            display(f'File: {fcName} delete = {result}')
        else:
            display(f'Did not delete: {fcName}')
    except:
        display(f'ERROR DELETING {fcName}')


# Make Selecttion and Create New Feature Class into function
def newFCfromSelect(GDBpath, selectFrom, selector, selType):
    try:
        switchSel = (input(f'Switch the resulting selection(y/N)? ')).upper()
        # set workspace GDB, overwrite = True
        arcpy.env.workspace,arcpy.env.overwriteOutput = GDBpath, True
        arcpy.Compact_management(GDBpath)
        # create random string to append to file name, prevents duplication errors
        rand = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 8))
        # tempLayer name from selectFrom name and random 8 character string
        # clear any previous selection and create temporary layer
        arcpy.SelectLayerByAttribute_management(selectFrom, "CLEAR_SELECTION")
        arcpy.MakeFeatureLayer_management(selectFrom, 'tempLayer') 
        # clear selection on selector and perfrom on temporary layer
        arcpy.SelectLayerByAttribute_management('tempLayer', "CLEAR_SELECTION")
        arcpy.SelectLayerByLocation_management('tempLayer', selType, selector)
        if switchSel == 'Y':
            arcpy.SelectLayerByAttribute_management('tempLayer', "SWITCH_SELECTION")
        # Check to see if any features were selected
        matchcount = int(arcpy.GetCount_management('tempLayer')[0]) 
        if matchcount == 0:
            print('no features matched spatial and attribute criteria')
        else:
            arcpy.CopyFeatures_management('tempLayer', (f'{selectFrom}_{rand}_{selType}'))
            display(f'{matchcount} Features selected and written to: {selectFrom}_{rand}_{selType}')
    except:
        display(f'ERROR Creating Selection: {selectFrom} {selType} {selector}')
        
def setCRS(envPath):
    try:
        test, count = 1, 0
        while test != 0:    
            dirList,layList = [], []
            arcpy.env.workspace = envPath
            arcpy.env.overwriteOutput = True
            print(f'ArcPy Workspace: {arcpy.env.workspace}')
            os.chdir(arcpy.env.workspace)
            print(f'Subdirectories')
            for i, item in enumerate(os.scandir(envPath)):
                if item.is_dir():
                    dirList.append(item.path)
                    print(f'{i} - {item.path}')
            ind = int(input(f'Change to Sub-Directory?<ENTER for none> ') or 999)
            if ind != 999:
                arcpy.env.workspace = dirList[ind]
                print(f'ArcPy Workspace: {arcpy.env.workspace}')
            for i, item in enumerate(arcpy.ListFeatureClasses()):
                layList.append(item)
                print(f'{i} - {item}')
            ind = int(input(f'TARGET LAYER TO DEFINE PROJECTION: ') or 999)
            if ind != 999:
                CRS = arcpy.SpatialReference(4326)
                arcpy.DefineProjection_management(layList[ind],CRS)
                print(f'Successfully Defined Projection for {layList[ind]}')
                count += 1
            test = int(input(f'PRESS 1 TO CONTINUE? ') or 0)
        print(f'Set CRS for {count} layers')
    except:
        print(f'ERROR Defining Projection')

In [66]:
class CostPathFromSurface:
    def __init__(self, workSpaceGDB: str, surfaceFile: str, originLoc: str, destLoc: str, siteName: str,
                 inPrj: str, outPrj: str, surfaceAttr: str, rastCellSize: str, ):
        self.workSpaceGDB,self.surfaceFile,self.originLoc,self.destLoc,self.siteName = workSpaceGDB,surfaceFile,originLoc,destLoc,siteName
        self.inPrj,self.outPrj,self.surfaceAttr,self.rastCellSize = inPrj, outPrj, surfaceAttr, rastCellSize
        self.timeSuffix, self.resamplingType, self.prjRastCellSize = time.strftime('%d%m%Y%H%M%S'), 'NEAREST', '45.6326806908288 45.6326806908286'
        # setup arcpy environment
        arcpy.env.workspace, arcpy.env.overwriteOutput = self.workSpaceGDB, True
        self.scratch = arcpy.env.scratchGDB
        # Full function workflow for route generation
        self.surfaceVecToRast()
        self.projectRaster()
        self.projectLocations()
        self.leastCostPath()

    # Convert surface vector into raster format - self.rasterSurface
    def surfaceVecToRast(self):
        self.rasterSurface = (f'{self.siteName}_rasterSurface_{timeSuffix}')
        arcpy.conversion.PolygonToRaster(self.surfaceFile, self.surfaceAttr, self.rasterSurface, cell_assignment="CELL_CENTER", 
                                         priority_field="NONE", cellsize=self.rastCellSize, build_rat="BUILD")

    # Project surface raster
    def projectRaster(self):
        self.rastSurfProjName = (f'{self.siteName}_rasterSurfaceProject_{self.timeSuffix}')
        self.rasterSurfaceProject = arcpy.management.ProjectRaster(self.rasterSurface, self.rastSurfProjName, out_coor_system=self.outPrj, 
            resampling_type=self.resamplingType, cell_size=self.prjRastCellSize, geographic_transform=[], Registration_Point="", 
            in_coor_system="", vertical="NO_VERTICAL") 
            
    # Project origin and destination vector points
    def projectLocations(self):
        self.originProj = arcpy.management.Project(self.originLoc, (f'{self.originLoc}_orjProj'), out_coor_system=self.outPrj, transform_method=[], 
            in_coor_system=self.inPrj, preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
        self.destProj = arcpy.management.Project(self.destLoc, (f'{self.destLoc}_orjProj'), out_coor_system=self.outPrj, transform_method=[], 
            in_coor_system=self.inPrj, preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
    
    # And finally generate leastCostPath!
    def leastCostPath(self):
        with arcpy.EnvManager(scratchWorkspace=self.scratch, workspace=self.scratch):
            arcpy.intelligence.LeastCostPath(self.rasterSurfaceProject, self.originProj, self.destProj, 
            (f'{self.workSpaceGDB}\\{self.siteName}_leastCostPath'), "SMALL_POSITIVE")


In [67]:
workSpaceGDB = 'C:\\Users\\Eric Kerney\\arcgisNotebooks\\leastCost\\Surface_v2.gdb\\Surface.gdb'
surfaceFile, originLoc, destLoc, siteName = 'PendletonOR_SuitabilitySurface', 'TribalCenter', 'Pendleton_Lab', 'pendleOR'
inPrj = 'GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]'
outPrj = 'PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]'
surfaceAttr, rastCellSize = 'score_v2', '0.0004'
costSur = CostPathFromSurface(workSpaceGDB, surfaceFile, originLoc, destLoc, siteName, inPrj, outPrj, surfaceAttr, rastCellSize)


In [62]:
costSur.surfaceVecToRast()

In [63]:
costSur.projectRaster()

In [64]:
costSur.projectLocations()

In [65]:
costSur.leastCostPath()

In [54]:
workPath = listWorkSpaces('C:\\Users\\Eric Kerney\\arcgisNotebooks\\leastCost\\Surface_v2.gdb')
wrkSpace = workPath[0]

0 - C:\Users\Eric Kerney\arcgisNotebooks\leastCost\Surface_v2.gdb\Surface.gdb


In [49]:
#Set workspace and variables 
arcpy.env.workspace=(workPath[0])
arcpy.env.overwriteOutput = True 
scratch = arcpy.env.scratchGDB

outCostPathName = 'leastCostPath'
siteName = "pendle_" #used for file naming 
timeSuffix = time.strftime("%d%m%Y%H%M%S") #used for file naming
#simpTol = "300 Meters"
prj = 'PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]'
wgs84Prj = 'GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]'
prjRastCellSize = '45.6326806908288 45.6326806908286'
resamplingType ="NEAREST"

originLoc = "TribalCenter"
destLoc = "Pendleton_Lab"

rastCellSize="0.0004"

print(timeSuffix)

11032022141448


In [50]:
#Convert polygon surface to Raster Surface
inputSurface = 'PendletonOR_SuitabilitySurface'
rasterSurface = (f'{siteName}_rasterSurface_{timeSuffix}')
arcpy.conversion.PolygonToRaster(inputSurface, "score_v2", rasterSurface, cell_assignment="CELL_CENTER", priority_field="NONE", 
                                 cellsize=rastCellSize, build_rat="BUILD")

<Result 'C:\\Users\\Eric Kerney\\arcgisNotebooks\\leastCost\\Surface_v2.gdb\\Surface.gdb\\pendle__rasterSurface_11032022141448'>

In [12]:
# LeastCostPath = "LeastCostPath"
originProj = arcpy.management.Project(originLoc, (f'{originLoc}_orjProj'), out_coor_system=prj, transform_method=[], in_coor_system=wgs84Prj, 
    preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
destProj = arcpy.management.Project(destLoc, (f'{destLoc}_destProj'), out_coor_system=prj, transform_method=[], in_coor_system=wgs84Prj, 
    preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")

In [14]:
#Projet Raster 
rastSurfProjName = (f'{siteName}_rasterSurfaceProject_{timeSuffix}')
rasterSurfaceProject = arcpy.management.ProjectRaster(rasterSurface, rastSurfProjName, out_coor_system=prj, resampling_type=resamplingType, 
    cell_size=prjRastCellSize, geographic_transform=[], Registration_Point="", in_coor_system="", vertical="NO_VERTICAL")

In [15]:
# originProj - Output path to projected origin location
# destProj - Output path to projected destination location
# rasterSurfaceProject - Output from projected surface raster
# scratch - scrath GDB workspace defined above = arcpy.env.scratchGDB 

with arcpy.EnvManager(scratchWorkspace=scratch, workspace=scratch):
    arcpy.intelligence.LeastCostPath(rasterSurfaceProject, originProj, destProj, (f'{wrkSpace}\\{siteName}{outCostPathName}'), "SMALL_POSITIVE")

In [6]:
arcpy.env.scratchGDB

'C:\\Users\\ERICKE~1\\AppData\\Local\\Temp\\scratch.gdb'

In [150]:
### YES!
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Eric Kerney\Documents\ArcGIS\Projects\MyProject\MyProject.gdb", 
workspace=r"C:\Users\Eric Kerney\Documents\ArcGIS\Projects\MyProject\MyProject.gdb"):
    arcpy.intelligence.LeastCostPath(
    r"C:\Users\Eric Kerney\arcgisNotebooks\leastCost\Surface_v2.gdb\Surface.gdb\Pendleton_OR__rasterSurfaceProject_11032022115239", 
    r"C:\Users\Eric Kerney\arcgisNotebooks\leastCost\Surface_v2.gdb\Surface.gdb\TribalCenter_Origin_Project_", 
    r"C:\Users\Eric Kerney\arcgisNotebooks\leastCost\Surface_v2.gdb\Surface.gdb\Pendleton_Lab_Dest_Project_", 
    r"C:\Users\Eric Kerney\arcgisNotebooks\leastCost\Surface_v2.gdb\Surface.gdb\leastCostTest1", "SMALL_POSITIVE")

## NEXT SECTION

In [7]:
#Create Barrier
Barrier, Count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=PendletonOR_Suitability_Experiment_3m_Buffer, selection_type="NEW_SELECTION", where_clause="score_v2 > 4", invert_where_clause="")

In [8]:
#Simplify Line
Simplify_Line = String+"SimplifyLine_"+Value
Simplify_Line_Output = arcpy.cartography.SimplifyLine(in_features=Path_output, out_feature_class=Simplify_Line, algorithm="WEIGHTED_AREA", tolerance=Simplification_Tolerance, error_resolving_option="RESOLVE_ERRORS", collapsed_point_option="NO_KEEP", error_checking_option="CHECK", in_barriers=[Barrier])[0]

In [9]:
#Generalize Line Further 
Generalize_Line_Output = arcpy.edit.Generalize(in_features=Simplify_Line_Output, tolerance="100 Meters")[0]

In [14]:
out_json = String+"_toGeoJSON_"+Value
GeoJson_Output = arcpy.FeaturesToJSON_conversion(in_features =Generalize_Line_Output, out_json_file = out_json, format_json="FORMATTED", geoJSON="GEOJSON")